# Score prospects in real time using Snowplow Signals and machine learning


Welcome to the [Snowplow Signals](https://docs.snowplow.io/docs/signals/) real-time prospect scoring [tutorial](https://docs.snowplow.io/tutorials/signals-ml-prospect-scoring/intro).

This notebook is intended to be used as part of the tutorial. It's hosted in Google Colab so you won't need to configure anything locally.

Check out the tutorial pages to follow along.

## Set up the notebook

This notebook uses Signals Sandbox connection. If you use Signals in the Console uncomment relevant lines.

Add your credentials to the Colab notebook secrets. Or just replace these variable values.

In [ ]:
from google.colab import userdata

# Signals Sandbox
SP_SANDBOX_URL = userdata.get('SP_SANDBOX_URL')     # https://{{123abc}}.svc.snplow.net
SP_SANDBOX_TOKEN = userdata.get('SP_SANDBOX_TOKEN') # 12345678-0000-1111-2222-123456789012

# Uncomment for Signals in the Console
# SP_API_URL = userdata.get('SP_API_URL')       # Signals API URL
# SP_API_KEY = userdata.get('SP_API_KEY')       # Signals API key
# SP_API_KEY_ID = userdata.get('SP_API_KEY_ID') # Signals API key ID
# SP_ORG_ID = userdata.get('SP_ORG_ID')         # Snowplow org ID

## Install notebook dependencies

In [ ]:
%pip install snowplow-signals

### Connect to Signals

In [ ]:
from snowplow_signals import Signals, SignalsSandbox, Event
from datetime import timedelta

sp_signals = SignalsSandbox(
    api_url=SP_SANDBOX_URL,
    sandbox_token=SP_SANDBOX_TOKEN,
)

# Uncomment if you use Signals in the Console
# sp_signals = Signals(
#     api_url=SP_API_URL,
#     api_key=SP_API_KEY,
#     api_key_id=SP_API_KEY_ID,
#     org_id=SP_ORG_ID
# )

### Define attributes

In [ ]:
from snowplow_signals import Attribute, Criteria, Criterion, AtomicProperty, EntityProperty

# Define an event
sp_page_view = Event(
    vendor="com.snowplowanalytics.snowplow",
    name="page_view",
    version="1-0-0"
)

# Define attributes
num_page_views = Attribute(
    name="num_page_views",
    type="int32",
    events=[sp_page_view],
    aggregation="counter"
)

num_pricing_views = Attribute(
    name="num_pricing_views",
    type="int32",
    events=[sp_page_view],
    aggregation="counter",
    criteria=Criteria(
        all=[
            Criterion.like(
                AtomicProperty(name="page_url"),
                "%pricing%"
            )
        ]
    )
)

num_customers_views = Attribute(
    name="num_customers_views",
    type="int32",
    events=[sp_page_view],
    aggregation="counter",
    criteria=Criteria(
        all=[
            Criterion.like(
                AtomicProperty(name="page_url"),
                "%customers%"
            )
        ]
    )
)

### Define an attribute group and a service

In [ ]:
from snowplow_signals import StreamAttributeGroup, domain_userid, Service

user_attributes_group = StreamAttributeGroup(
    name='prospect_scoring_tutorial',
    owner='your_email@example.com',
    version=1,
    attribute_key=domain_userid,
    attributes=[
        num_page_views,
        num_pricing_views,
        num_customers_views,
    ],
)

prospect_scoring_tutorial_service = Service(
    name='prospect_scoring_tutorial_service',
    owner='your_email@example.com',
    attribute_groups=[user_attributes_group],
)

**Note:** if you're using Signals in the Console, you can test attributes on the last 1 hour of data like this:

```python
sp_signals.test(attribute_group=user_attributes_group, app_ids=["website"])
```

### Deploy configuration to Signals

Running this cell will connect to your Signals instance. Signals will start calculating attributes to populate your Profiles Store.

In [ ]:
published = sp_signals.publish([user_attributes_group, prospect_scoring_tutorial_service])

# This should print "2 objects published"
print(f"{len(published)} objects published")

**Note:** you can't publish the same version of the attribute group or service a couple times. You will need to increase the version number or unpublish+delete the existing versions. You can use the code below during the tutorial to reset the attribute group and service after publishing them.


```python
sp_signals.unpublish([prospect_scoring_tutorial_service])
sp_signals.unpublish([user_attributes_group])
sp_signals.delete([prospect_scoring_tutorial_service])
sp_signals.delete([user_attributes_group])
```

### Look at your attributes

Use the Snowplow Inspector browser plugin to find your current `domain_userid`.

In [ ]:
sp_signals_result = sp_signals.get_service_attributes(
    name="prospect_scoring_tutorial_service",
    attribute_key="domain_userid",
    identifier="00000000-1111-2222-3333-444455556666", # UPDATE THIS
)
sp_signals_result

## Train the ML prospect scoring model

### Prepare historical dataset

The code in this cell uses a sample of the Snowplow atomic table in a CSV format. Update this cell to connect to your warehouse and train on your own data. Duckdb SQL syntax is easily translatable to the common warehouses.

In [ ]:
csv_filename = 'https://github.com/snowplow-incubator/signals-notebooks/raw/refs/heads/main/web/sample_events.csv.gz'

query = f"""
	with
	-- Change to your events table
	events as (
	    select * from read_csv_auto('{csv_filename}')
	    where app_id = 'website' -- filter for your app_ids
	        and domain_userid is not null -- filter to traffic with domain_userids
	        and event_name in ('page_view', 'submit_form') -- filter the events you need
	        -- and derived_tstamp >= dateadd('day', -90, current_date) -- filter by time
	        -- filter out bots, new/returning users, etc
	),
	-- Filter out post-conversion events
	post_conv as (
		select
			domain_userid,
			min(derived_tstamp) as first_conv
		from events
		where event_name = 'submit_form'
		group by 1
	),
	-- Prepare the target column
	targets_as_of_event as (
	    -- Target: will this person 'submit_form' in the next 1 hour?
	    select
	        -- identifiers
	        er.event_id,
	        -- target
	        count_if(ef.event_name = 'submit_form') > 0 as target_had_submit_form_next1h,
	    from events er
	    left join events ef on er.domain_userid = ef.domain_userid
	        and er.derived_tstamp < ef.derived_tstamp -- only future events
	        and datediff('second', er.derived_tstamp, ef.derived_tstamp) <= 60 * 60 -- only the next 1h of events
	    group by er.event_id
	),
	-- Prepare training features and dataset
	final_training as (
	    select
			er.domain_userid,
	        er.domain_sessionid,
	        er.derived_tstamp,
	        er.event_name,
	        coalesce(count_if(eh.event_name = 'page_view'), 0) as num_page_views,
	        coalesce(count_if(eh.event_name = 'page_view' and eh.page_url like '%pricing%'), 0) as num_pricing_views,
	        coalesce(count_if(eh.event_name = 'page_view' and eh.page_url like '%customers%'), 0) as num_customers_views,
	        coalesce(t.target_had_submit_form_next1h, false) as target_had_submit_form_next1h,
	        p.domain_userid is not null as remove_as_post_conv,
	    from events er
	    left join events eh on er.domain_userid = eh.domain_userid
	        and eh.derived_tstamp < er.derived_tstamp
	    left join targets_as_of_event t on er.event_id = t.event_id
	    left join post_conv p on er.domain_userid = p.domain_userid
	    	and er.derived_tstamp >= p.first_conv
	    group by all
      having not p.domain_userid is not null -- remove post-conversion events
	)
	select * from final_training
"""

import duckdb
import pandas as pd

conn = duckdb.connect(database=':memory:')
db_df = conn.sql(query).df()
conn.close()

db_df

### Train the model

In [ ]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# preprocessing
x_columns = [
    'num_page_views',
    'num_pricing_views',
    'num_customers_views',
]
y_column = 'target_had_submit_form_next1h'

seed = 42
model = LogisticRegression(random_state=seed, class_weight='balanced')

# Split, Train, and Evaluate
X = db_df[x_columns]
y = db_df[y_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Export
joblib.dump(model, "model.joblib")

### Evaluate model performance

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss, ConfusionMatrixDisplay, roc_curve, classification_report
import matplotlib.pyplot as plt

# Quick Model Metric Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC: ", roc_auc_score(y_test, y_prob))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Visualize Confusion Matrix and ROC Curve
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, cmap='Blues', ax=axes[0])
axes[0].set_title("Confusion Matrix")
fpr, tpr, _ = roc_curve(y_test, y_prob)
axes[1].plot(fpr, tpr, label=f'ROC AUC: {roc_auc_score(y_test, y_prob):.2f}')
axes[1].plot([0, 1], [0, 1], 'k--')
axes[1].set_xlabel("False Positive Rate")
axes[1].set_ylabel("True Positive Rate")
axes[1].legend()
axes[1].set_title("ROC Curve")

plt.tight_layout()
plt.show()

## Create ML API endpoint and Score Prospects

### Create and serve an API endpoint

For simplicity we use free TryCloudflare tunnels to expose this Colab notebook behind an HTTPS endpoint.

When this cell runs, you should see your URL like `https://abc-123.trycloudflare.com` in the output.


In [ ]:
%pip install flask-cloudflared

In [ ]:
from flask import Flask, request
from flask_cloudflared import run_with_cloudflared

# Define and launch Colab API Proxy
app = Flask(__name__)
run_with_cloudflared(app) # Open Cloudflared demo tunnel

# Load model
model = joblib.load("model.joblib")

# Process the Signals data for individual domain_userids
def get_duid_values(duid: str):
    # Retrieve attributes from Signals
    response = sp_signals.get_service_attributes(
        name="prospect_scoring_tutorial_service",
        attribute_key="domain_userid",
        identifier=duid,
    )
    signals_df = pd.DataFrame([response])
    # Prepare ML dataframe
    ml_df = signals_df.fillna(0).reindex(columns=x_columns, fill_value=0)
    return (signals_df, ml_df)

# Score the data against the model
def get_predictions(df):
    return float(model.predict_proba(df)[:, 1][0])

@app.route("/predict", methods=['POST'])
def predict():
    input_dict = request.get_json() # Parse JSON input

    # Get Signals data and prepare dataframe for scoring
    signals_df, ml_df = get_duid_values(input_dict['domain_userid'])

    # Score dataframe using the trained model
    prediction = get_predictions(ml_df)

    # Return the result
    print(f"P: {round(prediction, 4)} - {input_dict}")
    return {
        "signals": signals_df.to_dict(orient='records')[0],
        "scoring_attributes": ml_df.to_dict(orient='records')[0],
        "score": prediction
    }

app.run()

### Test using cURL

Verify via cURL, using the ngrok URL and your current `domain_userid` on your website:

```bash
curl -X POST "https://<YOUR_CLOUDFLARED_URL>.trycloudflare.com/predict" \
-H "Content-Type: application/json" \
-d '{"domain_userid": "UPDATE THIS"}'
```

### See scores in the browser

Verify via JS in the browser console:

```js
let api_url = "https://<UPDATE_THIS>.trycloudflare.com/predict"; // UPDATE THIS
let tracker_name = "sp"; // MAYBE UPDATE THIS

// Calls the API every 10s from the front-end
setInterval(function () {
    // assuming the Snowplow tracker is available at 'window.snowplow(...)'
    window.snowplow(function () {
        // Gets domain_userid from the tracker instance
        var sp = this[tracker_name];
        var domainUserId = sp.getDomainUserId();

        // Calls the API
        fetch(api_url, {
            method: "POST",
            headers: { "Content-Type": "application/json" },
            body: JSON.stringify({ domain_userid: domainUserId })
        })
        .then(response => response.json())
        .then(result => {
            console.log("Prediction: ", domainUserId, " - ", result.score);

            // Acts on prediction
            if (result.score >= 0.8) console.log('Prospect is likely to convert!');
        })
        .catch(console.error);
    });
}, 10 * 1000);
```